In [1]:
# Written by Kalon Overholt
# Young Lab
# 1/13/2022
# This script calculates binding isotherms for equilibrium protein-ligand interactions using a ligand depletion model

In [55]:
# Import libraries
import scipy as sp
from pylab import *
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import collections
from IPython.display import clear_output
import random
sns.set_style("ticks")
from scipy.optimize import curve_fit

In [3]:
# Define parameters
L=10 # Ligand (RNA) concentration (nM)

In [170]:
# Create function handle
def modelfun(x,b0,b1,b2):
    return b1+(b2-b1)*(L+x+b0-np.sqrt((L+x+b0)**2-4*L*x))/(2*L)

In [300]:
# Read in data
filename='220112_TF_motifDNA-analysis';
T=pd.read_csv(filename+'.csv',header=None,index_col=0)
proteins=pd.unique(T.index)

nrows=size(T,0)

i=0;
j=0;
k=3; # Number of replicates
n=len(proteins);

while i<nrows:

    P=T.iloc[i,:]
    P=P.to_numpy()
    data=T.iloc[i+1:i+1+k,:]
    Y=np.mean(data,axis=0)
    
    # Remove NANs
#     P(:,all(~isnan(P)));
#     Y(:,all(~isnan(Y)));
    
    # Fit model for fraction ligand bound
    # Modeled by anisotropy when ligand is fluorescent

    p0=[1000, 200, 400] #Define intial guess for the fit parameter for Kd
    popt, pcov = curve_fit(modelfun,P,Y,p0)
    
    i=i+k+1; # Skip to the next protein
    
    p_sim = np.logspace(-2,4)
    
    b0=popt[0]
    b1=popt[1]
    b2=popt[2]
    
    anisotropy=modelfun(p_sim,b0,b1,b2)    
    fb=(anisotropy-b1)/(b2-b1);
    data_fb=(data-b1)/(b2-b1);
    Y_fb=(Y-b1)/(b2-b1);

    # Plot fraction bound isotherms
    fig=plt.figure(figsize = (7,7))
    ax = plt.axes()
    P=P.transpose()
    ax.plot(np.log10(p_sim),fb,'-',color='black',linewidth=0.5) # Plot fit curve
    ax.plot(np.log10(P),data_fb.T,'.',color='black') # Plot data points
    ax.errorbar(np.log10(P),Y_fb,np.std(data_fb,ddof=1),ecolor='black',elinewidth=0.5,capsize=2,ls='none') # Plot error bars with ddof=1 for sample standard deviation
    ax.grid(False)
    ax.set_facecolor('white')
    ax.patch.set_edgecolor('black')  
    ax.patch.set_linewidth('2')     
    plt.xlabel('log([P]) (nM)')
    plt.ylabel('Fraction Bound')
    plt.xlim([-1, 4])
    plt.ylim([-0.2, 1.5])
    plt.title(proteins[j]+'\n Kd =  '+str(round(b0))+' nM')
    plt.show()
    
    ax.figure.savefig(filename+proteins[j]+'.pdf',bbox_inches='tight')
    
    # Plotting anisotropy
#         plotHandles(j)=plot(log10(p_sim),anisotropy,'-','Color',color);
#         hold on
#         plot(log10(P),data,'.','Color',color);
#         errorbar(log10(P), Y, std(data), 'LineStyle','none','Color',color);
#         xlabel('log([P]) (nM)','FontSize',14)
#         ylabel('Anisotropy','FontSize',14)
#         xlim([0 4])
#         ylim([100 300])
      
    j=j+1;

# If saving to one file
# legend(plotHandles,proteins,'Location','northeastoutside','FontSize',14);
# set(gcf, 'Position',  [100, 100, 700, 400])
# saveas(gcf,strcat(filename,'.pdf'))
